In [2]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for images, labels in train_dataloader:
    print(f"Shape of Images: {images.shape}")
    print(f"Shape of labels: {labels.shape}")
    break

Shape of Images: torch.Size([64, 1, 28, 28])
Shape of labels: torch.Size([64])


In [5]:
weird_shape = np.ndarray([4, 1, 2, 3])
print(f"Shape of weird: {weird_shape.shape}")
print(weird_shape)

Shape of weird: (4, 1, 2, 3)
[[[[6.95328155e-310 0.00000000e+000 4.94065646e-324]
   [8.56358748e-312 0.00000000e+000 0.00000000e+000]]]


 [[[8.56393190e-312 8.56333356e-312 8.56409754e-312]
   [1.48219694e-323 4.94065646e-324 1.38338381e-322]]]


 [[[1.38338381e-322 0.00000000e+000 0.00000000e+000]
   [4.94065646e-324 1.38338381e-322 4.94065646e-324]]]


 [[[0.00000000e+000 0.00000000e+000 0.00000000e+000]
   [3.87347466e-321 1.74918235e-309 1.25622152e-311]]]]


Set the default computing device.

In [6]:
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Device: {device}")

Device: cuda


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, training_data):
        training_data = self.flatten(training_data)
        logits = self.linear_relu_stack(training_data)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

How the training work
1. Use the model to do a prediction (at first it will be really bad because the ai is basically randomized)
2. Take the result and calculate the difference using the loss function
3. backpropagate the loss function result and optimize using the optimizer

In [10]:
def train(dataloader: torch.utils.data.DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss,
          optimizer: torch.optim.SGD):
    size = len(dataloader.dataset)
    model.train()
    for batch, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        
        # Compute prediction error
        prediction = model(images)
        loss = loss_fn(prediction, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(images) # 64 Images
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
                
train(train_dataloader, model, loss_fn, optimizer)

Loss: 2.308572 [   64/60000]
Loss: 2.295217 [ 6464/60000]
Loss: 2.279717 [12864/60000]
Loss: 2.265695 [19264/60000]
Loss: 2.246902 [25664/60000]
Loss: 2.216999 [32064/60000]
Loss: 2.223699 [38464/60000]
Loss: 2.189708 [44864/60000]
Loss: 2.187896 [51264/60000]
Loss: 2.149831 [57664/60000]


In [79]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
print(logits)
prediction_probabilities = nn.Softmax(dim=1)(logits)
print(prediction_probabilities)
label_prediction = prediction_probabilities.argmax(1)
print(f"Label Prediction: {label_prediction}")

tensor([[ 0.3250, -2.6991,  1.7266, -0.9593,  1.6036, -0.5673,  1.7334, -2.2809,
          2.2517, -0.4983]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0479, 0.0023, 0.1947, 0.0133, 0.1722, 0.0196, 0.1961, 0.0035, 0.3292,
         0.0210]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Label Prediction: tensor([8], device='cuda:0')


In [11]:
def test(dataloader: torch.utils.data.DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            prediction = model(images)
            test_loss += loss_fn(prediction, labels).item()
            correct += (prediction.argmax(1) == labels).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
Loss: 2.165370 [   64/60000]
Loss: 2.154741 [ 6464/60000]
Loss: 2.108157 [12864/60000]
Loss: 2.113154 [19264/60000]
Loss: 2.056176 [25664/60000]
Loss: 2.001973 [32064/60000]
Loss: 2.021013 [38464/60000]
Loss: 1.949157 [44864/60000]
Loss: 1.952057 [51264/60000]
Loss: 1.866928 [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.881707 

Epoch 2
Loss: 1.913760 [   64/60000]
Loss: 1.882868 [ 6464/60000]
Loss: 1.783390 [12864/60000]
Loss: 1.812737 [19264/60000]
Loss: 1.687658 [25664/60000]
Loss: 1.656894 [32064/60000]
Loss: 1.662443 [38464/60000]
Loss: 1.578928 [44864/60000]
Loss: 1.599371 [51264/60000]
Loss: 1.482323 [57664/60000]
Test Error: 
 Accuracy: 61.8%, Avg loss: 1.515086 

Epoch 3
Loss: 1.581485 [   64/60000]
Loss: 1.542271 [ 6464/60000]
Loss: 1.410966 [12864/60000]
Loss: 1.470271 [19264/60000]
Loss: 1.340251 [25664/60000]
Loss: 1.357245 [32064/60000]
Loss: 1.353370 [38464/60000]
Loss: 1.292534 [44864/60000]
Loss: 1.323751 [51264/60000]
Loss: 1.218976 [57664/60000]
Te

In [13]:
# Save Model
torch.save(model.state_dict(), "./model.pth")
print("Saved PyTorch model state to model.pth")

Saved PyTorch model state to model.pth


In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
    
with torch.no_grad():
    for i in range(len(test_data)):
        image, label = test_data[i][0].to(device), test_data[i][1]
        prediction = model(image)
        predicted, actual = classes[prediction[0].argmax(0)], classes[label]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Coat"
Predicted: "Coat", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Dress", Actual: "Dress"
Predicted: "Coat", Actual: "Coat"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Bag", Actual: "Bag"
Predicted: "T-shirt/top", Actual: "T-shirt/top"
Predicted: "Shirt", Actual: "Pullover"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Sneaker", Actual: "Ankle boot"
Predicted: "T

In [15]:
x = torch.tensor(1, dtype=torch.float)
x = torch.rand_like(x, dtype=torch.float)
x.item()

0.4395688772201538